In [3]:
import csv
import requests
from bs4 import BeautifulSoup
from tkinter import *
from tkinter import ttk
import requests
import csv
import json
import webbrowser
import tkinter as tk

API_key = "26b79da11d43455e86c5c0b3a1ea4c41" #enter unique API Key

def getUserEntry(): #a function that gets the information fron the entry and stores it into a seperate variable
    
    frame2 = Frame(root, highlightbackground="salmon", bg = "light salmon", highlightthickness=2, height =400, width =400) #creating a frame that will hold the radiobuttons for recipe options
    frame2.grid(row=6, column=1, pady=10,padx=20) #frame grid placement
    
    def ing_get():#creating a nested function that uses BeautifulSoup to scrape the html of the site
        recipe_ans = recipe_answer.get()
        temp_id = recipe_dict.get(recipe_ans) #where the id for each recipe is saved, needed to get to site below to scrape
        source = requests.get(f"https://api.spoonacular.com/recipes/{temp_id}/ingredientWidget?apiKey={API_key}").text #pulling text from site using id and api key an
        soup = BeautifulSoup(source, "lxml") #web scraping with BeautifulSoup

        big_section = soup.find("div", id = 'spoonacular-ingredient-vis-list')

        #creating segments of text on the widget - using labels as placeholders to make space around a segment of tex
        
        preing_label= Label(root, text="Here is your Shopping List!", font="bold", bg = "light yellow").grid(row=5,column=3, pady=10) #label to go above the grocery list textbox
        text_widget = tk.Text(height=15, width=50) #creating textbox
        text_widget.grid(row=6,column=3, pady=(0,0)) #placing textbox on the grid
        
        for i in big_section.find_all("div", class_="spoonacular-ingredient-list"):
            ingredient1 = i.find("div", class_= "spoonacular-amount spoonacular-us").text #scraping amount of an ingredient
            ingredient2 = i.find("div", class_= "spoonacular-name").text #scraping name of an ingredient
            joined = ingredient1 + " " + ingredient2 #formatting name and amount together 
            text_widget.insert(END, "- " + joined + "\n") #adding the text to the textbox as bullet points

        big_section2 = soup.find("div", class_ = "spoonacular-ingredients-menu")

        big_big_section2 = big_section2.find("div", style = "float:left;width:140px;margin-left:20px;line-height:24px")
        servings = big_big_section2.find("span", id = "spoonacular-serving-stepper-initial").text
        joined2 = "Servings: " + servings
        text_widget.insert(END, "- " + joined2 + "\n") #adding the text to the textbox as bullet points
        #serv_label = Label(root, text=joined2).pack()
        
        def callback(url): #function to set up a URL with more info about the reciple
            webbrowser.open_new_tab(url) #using webbrowser to open a new window from the GUI
        
        link = Label(root, text="Click here for more info on your selected meal",font=('Helveticabold', 15), fg="blue", bg="light yellow", cursor="hand2") #label function to hold teh text for the URL
        link.grid(row=10,column=3,padx=(0,0)) #packing our text
        link.bind("<Button-1>", lambda e:
        callback(f"https://api.spoonacular.com/recipes/{temp_id}/ingredientWidget?apiKey={API_key}")) #calling the function and using the link corresponding the recipe selected

    global recipe_input 
    recipe_input = user_entry1.get()
    
    recipe_dict = {} #creating empty dictionary to be added to as user enters desired recipe
    count = 1
    spoonacular = requests.get(f"https://api.spoonacular.com/recipes/complexSearch?query={recipe_input}&apiKey={API_key}") #accessing site using input and API key
    json_file = spoonacular.json()
    
    intext = json_file['results']

    for i in intext:
        recipe_scrape = i["title"] #scraping the title of a recipe associated with the user input
        id_scrape = i["id"] # scraping the id of a recipe based on user input recipe
        temp_dict = {recipe_scrape: id_scrape} #storing recipe title and id into a dictionary
        recipe_dict.update(temp_dict)
        
    name_list = list(recipe_dict.keys()) #storing the dictionary in a variable and converting to a list format
    
    result_label = Label(root, text="Here are your results! Please choose one!", bg="light yellow").grid(row=5,column=1, pady=10) #label to select an recipe option
    
    rad_counter = 6
    for i in name_list: #for loop to create radiobuttons based on the list of options we created (name_list)
        rad_button = Radiobutton(frame2, text=i, variable=recipe_answer, value=i, command=ing_get, bg="light salmon", width=40) # creating radio buttons from the user's input
        rad_button.grid(row=rad_counter, column=1)#radiobutton placement with grid
        rad_counter += 1

root = Tk() #root widget
root.geometry("900x900") #set the window size
root.title("MealMap") #set the widget title
root.configure(background='light yellow') #set the background color

recipe_answer = StringVar() #tkinter variable

intro_text = Label(root, text="Welcome to MealMap!", font=("Georgia","36", "bold"), fg='coral', bg = "light yellow", anchor=CENTER).grid(row=0,column=1,columnspan=3)
direction_text = Label(root, text="What meal are you looking to cook? We will generate a grocery list for it.", bg = "light yellow", fg='indian red', font=("Georgia","20","italic")).grid(row=1, column=1,columnspan=3)
user_entry1 = Entry(root,width=40) #creates text box for user input
user_entry1.grid(row=2,column=1,pady=5,columnspan=3) #packaging user entry onto widget

ttk.Button(root, text= "Search", command= getUserEntry).grid(row=3,column=1,pady=5,columnspan=3) #creates a button that starts the getUserEntry function

root.mainloop()